In [ ]:
%%capture
# Install the latest Tensorflow version.
!pip install tensorflow_text
!pip install bokeh
!pip install simpleneighbors[annoy]
!pip install tqdm
!pip install unidecode

In [ ]:
import os,re,collections
import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU,LSTM, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow.compat.v2 as tf
import pickle

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')
#nous allons extraire nos données qui se trouvent dans un fichier zip dans notre drive 
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/rebuilt.Cross-Language-Dataset-master.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
#nous allons experimenter une partie de nos donnée dans cette partie on commence par le corpus APR 
English_Dataset = tf.data.Dataset.list_files("/tmp/Cross-Language-Dataset-master/dataset/chunks/APR/en/*.txt")
Frensh_Dataset = tf.data.Dataset.list_files("/tmp/Cross-Language-Dataset-master/dataset/chunks/APR/fr/*.txt")


In [ ]:
#nous allons organiser nos fichier text de sort a ce que les fichier en francais aille leur traduction anglaise
English_DatasetFiles=list(English_Dataset.as_numpy_iterator())
English_DatasetFiles.sort()
Frensh_DatasetFiles=list(Frensh_Dataset .as_numpy_iterator())
Frensh_DatasetFiles.sort()

In [ ]:
#cette fonction permet d'extraire extrait les donnees des fichier
def load_data(Files):
  sentences=[]
  for file_content in Files: 
    with open(file_content, "r") as f:
      data = f.read()
      sentences.append(data)
  return sentences

english_sentences=load_data(English_DatasetFiles)
french_sentences=load_data(Frensh_DatasetFiles)


In [ ]:
for i in range(5):
  print('Sample :',i)
  print(english_sentences[i])
  print(french_sentences[i])
  print('-'*50)

Sample : 0
the association gregory lemarchal

l' association gregory lemarchal

--------------------------------------------------
Sample : 1
after the dresden file

après les dossier dresden

--------------------------------------------------
Sample : 2
the tradition of lanfeust troy
invention of reverse auctions

la tradition lanfeust de troy
invention des ventes aux enchères

--------------------------------------------------
Sample : 3
the manner of dos passos

la manière de dos passos

--------------------------------------------------
Sample : 4
trace of fat in this book

trace de gras dans le livre

--------------------------------------------------


In [ ]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('English Vocab:',len(english_words_counter))
print('French Vocab:',len(french_words_counter))

English Vocab: 4036
French Vocab: 4309


In [ ]:
def tokenize(x):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(x)
  return tokenizer.texts_to_sequences(x), tokenizer

In [ ]:
def pad(x, length=None):
  return pad_sequences(x, maxlen=length, padding='post')

In [ ]:
def preprocess(x, y):
    """
      Prétraiter x et y
    : param x: Feature Liste des phrases
    : param y: Label Liste des phrases
    : return: Tuple de (pré-traité x, pré-traité y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # La fonction sparse_categorical_crossentropy de Keras nécessite que les étiquettes soient en 3 dimensions
    # Extension des dimensions
    
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [ ]:
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

In [ ]:
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 44
Max French sentence length: 46
English vocabulary size: 4020
French vocabulary size: 4294


In [ ]:
def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

   # Nous prédisons la sortie pour un mot donné, puis sélectionnons la meilleure réponse
   # En sélectionnant cette étiquette, nous énumérons le mot à partir de l'identifiant
    
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
     Créer et entraîner un modèle RNN à l'aide de l'incorporation de mots sur x et y
     
     : param input_shape: Tuple de la forme d'entrée
     : param output_sequence_length: Longueur de la séquence de sortie
     : param english_vocab_size: Nombre de mots anglais uniques dans le jeu de données
     : param french_vocab_size: Nombre de mots français uniques dans le jeu de données
     : return: modèle Keras construit, mais pas entraîné
     """
    # Hyperparamètres
    learning_rate = 0.005
    
    # Couches
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(LSTM(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 
     # Compilation du modèle
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [ ]:
# Le reshaping l'entrée pour travailler avec un RNN de base

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

In [ ]:
simple_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

In [ ]:
simple_rnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 46, 256)           1029376   
_________________________________________________________________
lstm_1 (LSTM)                (None, 46, 256)           525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 46, 1024)          263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 46, 1024)          0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 46, 4295)          4402375   
Total params: 6,220,231
Trainable params: 6,220,231
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

Epoch 1/20
2/2 [==============================] - 52s 17s/step - loss: 8.2919 - accuracy: 0.1518 - val_loss: 2.6137 - val_accuracy: 0.8253
Epoch 2/20
2/2 [==============================] - 49s 16s/step - loss: 2.3835 - accuracy: 0.8388 - val_loss: 1.8763 - val_accuracy: 0.8253
Epoch 3/20
2/2 [==============================] - 49s 16s/step - loss: 1.7054 - accuracy: 0.8397 - val_loss: 2.0693 - val_accuracy: 0.8253
Epoch 4/20
2/2 [==============================] - 48s 16s/step - loss: 1.8681 - accuracy: 0.8358 - val_loss: 1.9541 - val_accuracy: 0.8242
Epoch 5/20
2/2 [==============================] - 48s 16s/step - loss: 2.1041 - accuracy: 0.7132 - val_loss: 1.7931 - val_accuracy: 0.8271
Epoch 6/20
2/2 [==============================] - 49s 16s/step - loss: 1.4936 - accuracy: 0.8417 - val_loss: 1.6038 - val_accuracy: 0.8261
Epoch 7/20
2/2 [==============================] - 49s 16s/step - loss: 1.3480 - accuracy: 0.8415 - val_loss: 1.4944 - val_accuracy: 0.8390
Epoch 8/20
2/2 [===========

In [ ]:
def final_predictions(text):
  y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
  y_id_to_word[0] = '<PAD>'

  sentence = [english_tokenizer.word_index[word] for word in text.split()]
  sentence = pad_sequences([sentence], maxlen=preproc_french_sentences.shape[-2], padding='post')
  
  print(sentence.shape)
  print(logits_to_text(simple_rnn_model.predict(sentence[:1])[0], french_tokenizer))

In [ ]:
simple_rnn_model.save('model')

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [ ]:
! mkdir "pickles"

In [ ]:
with open('pickles/french_tokenizer.pickle', 'wb') as handle:
    pickle.dump(french_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('pickles/english_tokenizer.pickle', 'wb') as handle:
    pickle.dump(english_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('pickles/preproc_french_sentences.pickle', 'wb') as handle:
    pickle.dump(preproc_french_sentences, handle, protocol=pickle.HIGHEST_PROTOCOL) 